In [9]:
import pandas as pd
import geopandas as gpd
from datetime import datetime

## Task1

In [27]:
vehicle['date'].dtype

dtype('<M8[ns]')

In [29]:
#vehicle['date']= [pd.Timestamp(i,unit='s') for i in vehicle['Timestamp']]

In [114]:
requests = pd.read_csv("requests.csv")
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,1380949200,1149,725,218,3601
1,1380949200,1225,1714,2003,3601
2,1380949200,1237,3143,1314,2218
3,1380949200,1323,3566,1330,2573
4,1380949200,1328,2466,1323,2473


In [115]:
requests['Hour'] = requests['Timestamp'].apply(lambda x: pd.Timestamp(x,unit='s',tz='America/New_York').hour)

In [116]:
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour
0,1380949200,1149,725,218,3601,1
1,1380949200,1225,1714,2003,3601,1
2,1380949200,1237,3143,1314,2218,1
3,1380949200,1323,3566,1330,2573,1
4,1380949200,1328,2466,1323,2473,1


In [117]:
requests['Serving'] = requests['Actual_Dropoff'].apply(lambda x: 0 if x ==-1 else 1 )

In [118]:
unserving = requests[requests['Serving']==0]
serving = requests[requests['Serving']==1]

In [119]:
#requests['Hour']=requests['Hour'].astype('category')
#requests['Serving']=requests['Serving'].astype('category')

In [120]:
servinghours = serving.groupby('Hour').count()[['Serving']].reset_index()

In [121]:
hours = requests.groupby('Hour').count()[['Timestamp']].reset_index()

In [125]:
rate = hours.merge(servinghours, on='Hour')
rate.rename(columns={'Timestamp':'Total'}, inplace=True)
rate['rate']=rate['Serving']/rate['Total']
rate.head()

,Hour,Total,Serving,rate
0,0,19313,6748,0.349402
1,1,10052,7297,0.725925
2,2,6070,5464,0.900165
3,3,3725,3443,0.924295
4,4,2567,2310,0.899883


In [126]:
rate.to_csv('servingrate.csv')

## Task 2

In [129]:
geomanhattan = gpd.read_file('manhattan.geojson')
geomanhattan.head()

,id,w,geometry
0,0.0,NaN,POINT (-73.9975944 40.7140611)
1,1.0,NaN,POINT (-73.9980743 40.7137811)
2,2.0,NaN,POINT (-73.9615093 40.7602403)
3,3.0,NaN,POINT (-73.9692158 40.763488)
4,4.0,NaN,POINT (-73.9694382 40.7635824)


In [136]:
georequests = unserving.merge(geomanhattan, left_on='Requested_Pickup',right_on='id')

In [137]:
georequests.rename(columns={'geometry':'Geo_Pickup'}, inplace=True)

In [139]:
georequests = georequests.merge(geomanhattan, left_on='Requested_Dropoff',right_on='id')

In [141]:
georequests.rename(columns={'geometry':'Geo_Dropoff'}, inplace=True)

In [148]:
georequests.drop(['w_x','w_y'],axis=1,inplace=True)
georequests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour,Serving,id_x,Geo_Pickup,id_y,Geo_Dropoff
0,1380949380,1225,1279,-1,-1,1,0,1225.0,POINT (-73.972567 40.7450709),1279.0,POINT (-73.9498734 40.7761941)
1,1381025613,1225,1279,-1,-1,22,0,1225.0,POINT (-73.972567 40.7450709),1279.0,POINT (-73.9498734 40.7761941)
2,1381018306,1260,1279,-1,-1,20,0,1260.0,POINT (-73.9911207 40.7503112),1279.0,POINT (-73.9498734 40.7761941)
3,1381021560,3167,1279,-1,-1,21,0,3167.0,POINT (-73.9882441 40.7491343),1279.0,POINT (-73.9498734 40.7761941)
4,1381030809,229,1279,-1,-1,23,0,229.0,POINT (-73.9939536 40.7515176),1279.0,POINT (-73.9498734 40.7761941)


In [149]:
georequests.to_csv('georequests.csv')